In [ ]:
# loading packages
import numpy as np
import pandas as pd
from scipy.io import loadmat
import glob
import os
import sys
import matplotlib.pyplot as plt
from pingouin import pairwise_corr
import seaborn as sns
sns.set(style="white")

from scipy import stats
from mne.viz import circular_layout, plot_connectivity_circle
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

import scipy.stats as chi2_contingency
import scipy.stats as stats
from scipy.spatial import distance

# increasing display number of pandas columns and rows
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
# functions to transform vector to dfnc matrix
def calc_num_components(a):
    num_pairs = len(a)
    num_components = int(np.ceil(np.sqrt(2 * num_pairs - 1)))
    if num_components * (num_components - 1) == 2 * num_pairs:
        return num_components
    else:
        return 0


def return_dfnc_from_vector(a):
    '''This Function returns a dfnc matrix given a vector
        a : the vector of correlation pairs
    '''
    n = calc_num_components(a)
    out = np.zeros((n, n))

    if not n:
        return out
    
    l_indices = np.tril_indices(n, -1)
    u_indices = np.triu_indices(n, 1)

    out[u_indices] = a
    out[l_indices] = out.T[l_indices]
    
    return out

In [ ]:
# Load clinical variables of stroke patients
Salvo = pd.read_csv("/.../DFNC/Behavior/SALVO_1206.csv")

n_subjects = 41
# Number of dFNC windows, number of correlations
n_windows = 129
n_corr = 1176

# Defining subgroups of vairably affected patients
mild = Salvo.InHospitalNIHSS.sort_values()[0:19].index
moderate = Salvo.InHospitalNIHSS.sort_values()[19:34].index
severe = Salvo.InHospitalNIHSS.sort_values()[34:41].index

# load subject names
subject_names = glob.glob('/..../WithSM_dfnc_sub*',)
subject_names = sorted(subject_names)
# checked subject order

# load individual fncs, state information, fraction and dwell times and numbers of transition (as saved after dFNC computation in matlab)
dfnc = np.zeros((n_subjects,n_windows,n_corr))
for n in range(0,n_subjects,1):
   
    dfnc[n] = loadmat(subject_names[n])['FNCdyn']
    
x = loadmat('/.../states_stroke.mat')["a"]
for n in range(0,np.shape(x)[0],1):
    if n==0:
        states = pd.DataFrame(x[n][0])
    else:
        states[n] = pd.DataFrame(x[n][0])
states = states.transpose()

fraction = loadmat('/.../fraction_time.mat')["frac"]
fraction = pd.DataFrame(fraction)
fraction.columns = ["State_1", "State_2", "State_3"]

dwell = loadmat('/.../dwell_time.mat')["dwell"]
dwell = pd.DataFrame(dwell)
dwell.columns = ["State_1", "State_2", "State_3"]

num_trans = loadmat('/.../num_trans.mat')["num_trans"]
num_trans = pd.DataFrame(num_trans)

Subgroup analysis 

In [ ]:
# Three-level one-way ANOVA for fraction times 
a = stats.f_oneway(fraction["State_1"].iloc[mild], fraction["State_1"].iloc[moderate],fraction["State_1"].iloc[severe],)[1]
b = stats.f_oneway(fraction["State_2"].iloc[mild], fraction["State_2"].iloc[moderate],fraction["State_2"].iloc[severe],)[1]
c = stats.f_oneway(fraction["State_3"].iloc[mild], fraction["State_3"].iloc[moderate],fraction["State_3"].iloc[severe],)[1]
print("State 1: %f" %stats.f_oneway(fraction["State_1"].iloc[mild], fraction["State_1"].iloc[moderate],fraction["State_1"].iloc[severe],)[1])
print("State 2: %f" %stats.f_oneway(fraction["State_2"].iloc[mild], fraction["State_2"].iloc[moderate],fraction["State_2"].iloc[severe],)[1])
print("State 3: %f" %stats.f_oneway(fraction["State_3"].iloc[mild], fraction["State_3"].iloc[moderate],fraction["State_3"].iloc[severe],)[1])

In [ ]:
# post-hoc t-test 
n_dfnc = ["State_1", "State_2", "State_3"]
no = [0,1,2,]
p_all_better = np.zeros(3)
print("Fraction time: Mild vs Moderate")
for n, p in zip(n_dfnc, no):
    rvs1 = fraction[n].iloc[mild]
    rvs2 = fraction[n].iloc[moderate]
    p_all_better[p] = stats.ttest_ind(rvs1,rvs2)[1] 
    print("%s" % n)
    print("Moderate deficit: %f" %(rvs2.mean()))
    print("Mild deficit: %f" %(rvs1.mean()))
    print("%f" %stats.ttest_ind(rvs1,rvs2)[1])

In [ ]:
# post-hoc t-test 
p_all_worse = np.zeros(3)
print("Fraction time: Mild vs severe")
for n, p in zip(n_dfnc, no):
    rvs1 = fraction[n].iloc[mild]
    rvs2 = fraction[n].iloc[severe]
    p_all_worse[p] = stats.ttest_ind(rvs1,rvs2)[1] 
    print("%s" % n)
    print("Severe deficit: %f" %(rvs2.mean()))
    print("Mild deficit: %f" %(rvs1.mean()))
    print("%f" %stats.ttest_ind(rvs1,rvs2)[1])

In [ ]:
# post-hoc t-test Moderate vs severe
p_all_bw = np.zeros(3)
print("Fraction time: Patients with moderate vs Patients with severe symptoms")
for n, p in zip(n_dfnc, no):
    rvs1 = fraction[n].iloc[moderate]
    rvs2 = fraction[n].iloc[severe]
    p_all_bw[p] = stats.ttest_ind(rvs1,rvs2)[1] 
    print("%s" % n)
    print("Severe deficit: %f" %(rvs2.mean()))
    print("Moderate deficit: %f" %(rvs1.mean()))
    print("%f" %stats.ttest_ind(rvs1,rvs2)[1])

In [ ]:
# FDR-corrections for p-values in State 1
statsmodels.stats.multitest.multipletests([0.504671,0.004807, 0.050557], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)


In [ ]:
N = 3
All_severe = np.mean(fraction["State_1"].loc[severe]) + np.mean(fraction["State_2"].loc[severe]) + np.mean(fraction["State_3"].loc[severe])
All_moderate = np.mean(fraction["State_1"].loc[moderate]) + np.mean(fraction["State_2"].loc[moderate]) + np.mean(fraction["State_3"].loc[moderate])
All_mild = np.mean(fraction["State_1"].loc[mild]) + np.mean(fraction["State_2"].loc[mild]) + np.mean(fraction["State_3"].loc[mild])
State1Means = (np.mean(fraction["State_1"].loc[mild])/All_mild, np.mean(fraction["State_1"].loc[moderate])/All_moderate, np.mean(fraction["State_1"].loc[severe])/All_severe,)
State2Means = (np.mean(fraction["State_2"].loc[mild])/All_mild, np.mean(fraction["State_2"].loc[moderate])/All_moderate, np.mean(fraction["State_2"].loc[severe])/All_severe,)
State3Means = (np.mean(fraction["State_3"].loc[mild])/All_mild, np.mean(fraction["State_3"].loc[moderate])/All_moderate, np.mean(fraction["State_3"].loc[severe])/All_severe,)
bottom = np.add(State1Means, State2Means).tolist()


plt.figure(figsize=(14, 10))
ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = plt.barh(ind, State1Means, width,  color='darkgoldenrod', edgecolor='white')
p2 = plt.barh(ind, State2Means, width,
             left=State1Means, color='goldenrod', edgecolor='white')
p3 = plt.barh(ind, State3Means, width,
             left=bottom, color='gold', edgecolor='white')

plt.title('Fraction times', fontsize=25)
plt.yticks(ind, ('Mild Stroke Severity', 'Moderate Stroke Severity', "Severe stroke severity"), fontsize=15)
plt.xticks(fontsize=15)
#plt.savefig("ANOVA_Fraction.png", dpi=600)

In [ ]:
# Three-level one-way ANOVA for dwell times 
d = stats.f_oneway(dwell["State_1"].iloc[mild], dwell["State_1"].iloc[moderate],dwell["State_1"].iloc[severe],)[1]
e = stats.f_oneway(dwell["State_2"].iloc[mild], dwell["State_2"].iloc[moderate],dwell["State_2"].iloc[severe],)[1]
f = stats.f_oneway(dwell["State_3"].iloc[mild], dwell["State_3"].iloc[moderate],dwell["State_3"].iloc[severe],)[1]
print("State 1: %f" %stats.f_oneway(dwell["State_1"].iloc[mild], dwell["State_1"].iloc[moderate],dwell["State_1"].iloc[severe],)[1])
print("State 2: %f" %stats.f_oneway(dwell["State_2"].iloc[mild], dwell["State_2"].iloc[moderate],dwell["State_2"].iloc[severe],)[1])
print("State 3: %f" %stats.f_oneway(dwell["State_3"].iloc[mild], dwell["State_3"].iloc[moderate],dwell["State_3"].iloc[severe],)[1])

In [ ]:
# post-hoc t-test Control vs moderate
p_all_better = np.zeros(3)
print("Dwell time: Mild Moderate")
for n, p in zip(n_dfnc, no):
    rvs1 = dwell[n].iloc[mild]
    rvs2 = dwell[n].iloc[moderate]
    p_all_better[p] = stats.ttest_ind(rvs1,rvs2)[1] 
    print("%s" % n)
    print("Moderate deficit: %f" %(rvs2.mean()))
    print("Mild deficit: %f" %(rvs1.mean()))
    print("%f" %stats.ttest_ind(rvs1,rvs2)[1])

In [ ]:
# post-hoc t-test Control vs severe
p_all_worse = np.zeros(3)
print("Dwell time: Mild Severe")
for n, p in zip(n_dfnc, no):
    rvs1 = dwell[n].iloc[mild]
    rvs2 = dwell[n].iloc[severe]
    p_all_worse[p] = stats.ttest_ind(rvs1,rvs2)[1] 
    print("%s" % n)
    print("Severe deficit: %f" %(rvs2.mean()))
    print("Mild deficit: %f" %(rvs1.mean()))
    print("%f" %stats.ttest_ind(rvs1,rvs2)[1])

In [ ]:
# post-hoc t-test moderate vs severe
p_all_bw = np.zeros(3)
print("Dwell time: Moderate Severe")
for n, p in zip(n_dfnc, no):
    rvs1 = dwell[n].iloc[moderate]
    rvs2 = dwell[n].iloc[severe]
    p_all_bw[p] = stats.ttest_ind(rvs1,rvs2)[1] 
    print("%s" % n)
    print("Severe deficit: %f" %(rvs2.mean()))
    print("Moderate deficit: %f" %(rvs1.mean()))
    print("%f" %stats.ttest_ind(rvs1,rvs2)[1])

In [ ]:
# Correction for multiple comparison:  Dwell times State 1
statsmodels.stats.multitest.multipletests([0.820554,0.004715, 0.017701], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)


In [ ]:
N = 3
State1Means = (np.mean(dwell["State_1"].iloc[mild])/All_mild, np.mean(dwell["State_1"].iloc[moderate])/All_moderate, np.mean(dwell["State_1"].iloc[severe])/All_severe,)
State2Means = (np.mean(dwell["State_2"].iloc[mild])/All_mild, np.mean(dwell["State_2"].iloc[moderate])/All_moderate, np.mean(dwell["State_2"].iloc[severe])/All_severe,)
State3Means = (np.mean(dwell["State_3"].iloc[mild])/All_mild, np.mean(dwell["State_3"].iloc[moderate])/All_moderate, np.mean(dwell["State_3"].iloc[severe])/All_severe,)
bottom = np.add(State1Means, State2Means).tolist()


plt.figure(figsize=(14, 10))
ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = plt.barh(ind, State1Means, width,  color='darkgreen', edgecolor='white')
p2 = plt.barh(ind, State2Means, width,
             left=State1Means, color='g', edgecolor='white')
p3 = plt.barh(ind, State3Means, width,
             left=bottom, color='lightgreen', edgecolor='white')

#plt.ylabel('Scores')
plt.title('Dwell times', fontsize=25)

plt.yticks(ind, ('Mild Stroke Severity', 'Moderate Stroke Severity', "Severe stroke severity"), fontsize=15)
plt.xticks(fontsize=15)
plt.legend((p1[0], p2[0], p3[0]), ('State 1', 'State 2', 'State 3'), fontsize=15)
#plt.savefig("ANOVA_Dwell.png", dpi=600)
#plt.show()

In [ ]:
# Group differences in numbers of transition
rvs1 = num_trans.iloc[mild]
rvs2 = num_trans.iloc[moderate]
rvs3 = num_trans.iloc[severe]
print("Mean No of transitions")
print("No deficit: %f" %(rvs1.mean()))
print("Moderate Deficit: %f" %(rvs2.mean()))
print("Severe Deficit: %f" %(rvs3.mean()))
print("One-way ANOVA transition times: %f" %stats.f_oneway(rvs1, rvs2,rvs3,)[1])